In [2]:
import torch
from d2l import torch as d2l

基于GPU的并行运算 -- 云服务器(需要2个GPU)

In [8]:
devices = d2l.try_all_gpus()
def run(x):
    return [x.mm(x) for _ in range(50)]

x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])
x_gpu2 = torch.rand(size=(4000, 4000), device=devices[1])

IndexError: list index out of range

In [ ]:
run(x_gpu1)    # 预热设备，对设备执行一次传递，确保缓存的作用不影响最终结果
run(x_gpu2)
torch.cuda.synchronize(devices[0])
torch.cuda.synchronize(devices[1])

with d2l.Benchmark('GPU1 time'):
    run(x_gpu1)
    torch.cuda.synchronize(devices[0])  # devices[0] 表示是该设备需要同步

with d2l.Benchmark('GPU2 time'):
    run(x_gpu2)
    torch.cuda.synchronize(devices[1])  

In [ ]:
with d2l.Benchmark('GPU1 & GPU2'):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize()

并行运算与通信

In [3]:
def try_xpu(i=0):
    """如果存在,则返回gpu(i),否则返回cpu()"""
    if torch.xpu.device_count() >= i+1:
        return torch.device(f'xpu:{i}')
    return torch.device('cpu')

device = try_xpu()
def run(x):
    return [x.mm(x) for _ in range(50)]

x_xpu1 = torch.rand(size=(4000, 4000), device=device)

In [6]:
# 许多时候，需要在不同设备之间移动数据，例如执行分布式优化时，需要移动数据来聚合多个加速卡上的梯度
# 通过在GPU上计算，然后将结果复制会CPU来模拟这个过程
def copy_to_cpu(x, non_blocking=False):
    return [y.to('cpu', non_blocking=non_blocking) for y in x]

with d2l.Benchmark('在GPU1上运行'):
    y = run(x_xpu1)
    torch.xpu.synchronize()

with d2l.Benchmark('复制到CPU'):
    y_cpu = copy_to_cpu(y)
    torch.xpu.synchronize()

在GPU1上运行: 1.9619 sec
复制到CPU: 1.4300 sec


In [ ]:
# 系统可以在计算y[i]的同时复制y[i-1], 以减少总运行时间
with d2l.Benchmark('在GPU1上运行并复制到CPU'):
    y = run(x_xpu1)
    y_cpu = copy_to_cpu(y, True)
    torch.xpu.synchronize()

在GPU1上运行并复制到CPU: 2.4415 sec
